# Setup

In [ ]:
!pip install gradio

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import openai
import gradio as gr
import re
import time

In [ ]:
# %rm -rf /sdthings

In [ ]:
%cd gdrive/MyDrive

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/sdthings/scripts')
import random,os
from IPython import display as disp
from sdthings.scripts import tools
from os.path import basename
clear = tools.clear
basedir = '/content/gdrive/MyDrive/smooth_interpolation'
sd = tools.Sd(basedir, False)
args = sd.makeargs()

In [ ]:
#model_url = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt' 
model_url = 'https://huggingface.co/dreamlike-art/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.ckpt'
# %mkdir /content/gdrive/MyDrive/smooth_interpolation/models
model_name = basename(model_url)
# !wget {model_url} -O /content/gdrive/MyDrive/smooth_interpolation/models/{model_name}
clear()

In [ ]:
!nvidia-smi

In [ ]:
sd.load(model_name)
from sdthings.scripts.alivify import interpolate_prompts
clear()

# Example Inputs

In [ ]:
giving_tree_story = """
Once there was a tree....
and she loved a little boy.
And everyday the boy would come
and he would gather her leaves
and make them into crowns
and play king of the forest.
He would climb up her trunk
and swing from her branches
and eat apples.
And they would play hide-and-go-seek.
And when he was tired,
he would sleep in her shade.
And the boy loved the tree....
very much.
And the tree was happy.

But time went by.
And the boy grew older.
And the tree was often alone.
Then one day the boy came to the tree
and the tree said, "Come, Boy, come and
climb up my trunk and swing from my
branches and eat apples and play in my
shade and be happy."
"I am too big to climb and play" said
the boy.
"I want to buy things and have fun.
I want some money?"
"I'm sorry," said the tree, "but I
have no money.
I have only leaves and apples.
Take my apples, Boy, and sell them in
the city. Then you will have money and
you will be happy."
And so the boy climbed up the
tree and gathered her apples
and carried them away.
And the tree was happy.

But the boy stayed away for a long time....
and the tree was sad.
And then one day the boy came back
and the tree shook with joy
and she said, "Come, Boy, climb up my trunk
and swing from my branches and be happy."
"I am too busy to climb trees," said the boy.
"I want a house to keep me warm," he said.
"I want a wife and I want children,
and so I need a house.
Can you give me a house ?"
" I have no house," said the tree.
"The forest is my house,
but you may cut off
my branches and build a
house. Then you will be happy."
And so the boy cut off her branches
and carried them away
to build his house.
And the tree was happy.

But the boy stayed away for a long time.
And when he came back,
the tree was so happy
she could hardly speak.
"Come, Boy," she whispered,
"come and play."
"I am too old and sad to play,"
said the boy.
"I want a boat that will
take me far away from here.
Can you give me a boat?"
"Cut down my trunk
and make a boat," said the tree.
"Then you can sail away...
and be happy."
And so the boy cut down her trunk
and made a boat and sailed away.
And the tree was happy
... but not really.

And after a long time
the boy came back again.
"I am sorry, Boy,"
said the tree," but I have nothing
left to give you -
My apples are gone."
"My teeth are too weak
for apples," said the boy.
"My branches are gone,"
said the tree. " You
cannot swing on them - "
"I am too old to swing
on branches," said the boy.
"My trunk is gone, " said the tree.
"You cannot climb - "
"I am too tired to climb" said the boy.
"I am sorry," sighed the tree.
"I wish that I could give you something....
but I have nothing left.
I am just an old stump.
I am sorry...."
"I don't need very much now," said the boy.
"just a quiet place to sit and rest.
I am very tired."
"Well," said the tree, straightening
herself up as much as she could,
"well, an old stump is good for sitting and resting
Come, Boy, sit down. Sit down and rest."
And the boy did.
And the tree was happy."""

output_media_path = "/content/gdrive/MyDrive/givingtree.mov"

# Story Prompt New

In [ ]:
story_prompt = \
f"""Write a short children's book story about a friendship between a tree and a boy's friendship over the years as the boy takes more and more from the tree with a happy ending. Please write a story with 5 scenes proceeded by the SCENE number:

SCENE #1

Once there was a tree....
and she loved a little boy.
And everyday the boy would come
and he would gather her leaves
and make them into crowns
and play king of the forest.
He would climb up her trunk
and swing from her branches
and eat apples.
And they would play hide-and-go-seek.
And when he was tired,
he would sleep in her shade.
And the boy loved the tree....
very much.
And the tree was happy.

SCENE #2

But time went by.
And the boy grew older.
And the tree was often alone.
Then one day the boy came to the tree
and the tree said, "Come, Boy, come and
climb up my trunk and swing from my
branches and eat apples and play in my
shade and be happy."
"I am too big to climb and play" said
the boy.
"I want to buy things and have fun.
I want some money?"
"I'm sorry," said the tree, "but I
have no money.
I have only leaves and apples.
Take my apples, Boy, and sell them in
the city. Then you will have money and
you will be happy."
And so the boy climbed up the
tree and gathered her apples
and carried them away.
And the tree was happy.

SCENE #3

But the boy stayed away for a long time....
and the tree was sad.
And then one day the boy came back
and the tree shook with joy
and she said, "Come, Boy, climb up my trunk
and swing from my branches and be happy."
"I am too busy to climb trees," said the boy.
"I want a house to keep me warm," he said.
"I want a wife and I want children,
and so I need a house.
Can you give me a house ?"
" I have no house," said the tree.
"The forest is my house,
but you may cut off
my branches and build a
house. Then you will be happy."
And so the boy cut off her branches
and carried them away
to build his house.
And the tree was happy.

SCENE #4

But the boy stayed away for a long time.
And when he came back,
the tree was so happy
she could hardly speak.
"Come, Boy," she whispered,
"come and play."
"I am too old and sad to play,"
said the boy.
"I want a boat that will
take me far away from here.
Can you give me a boat?"
"Cut down my trunk
and make a boat," said the tree.
"Then you can sail away...
and be happy."
And so the boy cut down her trunk
and made a boat and sailed away.
And the tree was happy
... but not really.

SCENE #5

And after a long time
the boy came back again.
"I am sorry, Boy,"
said the tree," but I have nothing
left to give you -
My apples are gone."
"My teeth are too weak
for apples," said the boy.
"My branches are gone,"
said the tree. " You
cannot swing on them - "
"I am too old to swing
on branches," said the boy.
"My trunk is gone, " said the tree.
"You cannot climb - "
"I am too tired to climb" said the boy.
"I am sorry," sighed the tree.
"I wish that I could give you something....
but I have nothing left.
I am just an old stump.
I am sorry...."
"I don't need very much now," said the boy.
"just a quiet place to sit and rest.
I am very tired."
"Well," said the tree, straightening
herself up as much as she could,
"well, an old stump is good for sitting and resting
Come, Boy, sit down. Sit down and rest."
And the boy did.
And the tree was happy.

The End.

------

Write a short children's book story about {{character:}} living in {{location:}} {{situation:}} with a happy ending. Please write a story with {{num_scene:}} scenes proceeded by the SCENE number. Use the example above as a formatting and style reference:
"""

illustration_prompt = \
f"""SCENE #1

Once there was a tree....
and she loved a little boy.
And everyday the boy would come
and he would gather her leaves
and make them into crowns
and play king of the forest.
He would climb up her trunk
and swing from her branches
and eat apples.
And they would play hide-and-go-seek.
And when he was tired,
he would sleep in her shade.
And the boy loved the tree....
very much.
And the tree was happy.

SCENE #2

But time went by.
And the boy grew older.
And the tree was often alone.
Then one day the boy came to the tree
and the tree said, "Come, Boy, come and
climb up my trunk and swing from my
branches and eat apples and play in my
shade and be happy."
"I am too big to climb and play" said
the boy.
"I want to buy things and have fun.
I want some money?"
"I'm sorry," said the tree, "but I
have no money.
I have only leaves and apples.
Take my apples, Boy, and sell them in
the city. Then you will have money and
you will be happy."
And so the boy climbed up the
tree and gathered her apples
and carried them away.
And the tree was happy.

SCENE #3

But the boy stayed away for a long time....
and the tree was sad.
And then one day the boy came back
and the tree shook with joy
and she said, "Come, Boy, climb up my trunk
and swing from my branches and be happy."
"I am too busy to climb trees," said the boy.
"I want a house to keep me warm," he said.
"I want a wife and I want children,
and so I need a house.
Can you give me a house ?"
" I have no house," said the tree.
"The forest is my house,
but you may cut off
my branches and build a
house. Then you will be happy."
And so the boy cut off her branches
and carried them away
to build his house.
And the tree was happy.

SCENE #4

But the boy stayed away for a long time.
And when he came back,
the tree was so happy
she could hardly speak.
"Come, Boy," she whispered,
"come and play."
"I am too old and sad to play,"
said the boy.
"I want a boat that will
take me far away from here.
Can you give me a boat?"
"Cut down my trunk
and make a boat," said the tree.
"Then you can sail away...
and be happy."
And so the boy cut down her trunk
and made a boat and sailed away.
And the tree was happy
... but not really.

SCENE #5

And after a long time
the boy came back again.
"I am sorry, Boy,"
said the tree," but I have nothing
left to give you -
My apples are gone."
"My teeth are too weak
for apples," said the boy.
"My branches are gone,"
said the tree. " You
cannot swing on them - "
"I am too old to swing
on branches," said the boy.
"My trunk is gone, " said the tree.
"You cannot climb - "
"I am too tired to climb" said the boy.
"I am sorry," sighed the tree.
"I wish that I could give you something....
but I have nothing left.
I am just an old stump.
I am sorry...."
"I don't need very much now," said the boy.
"just a quiet place to sit and rest.
I am very tired."
"Well," said the tree, straightening
herself up as much as she could,
"well, an old stump is good for sitting and resting
Come, Boy, sit down. Sit down and rest."
And the boy did.
And the tree was happy.

The End.

Write a 10-155 word description of each scene in the story above. Refer to the main character as <main char>:

SCENE #1: a beautiful tree and <main char> standing next to the tree
SCENE #2: a beautiful tree and <main char> picking apples from a ladder
SCENE #3: a beautiful tree and <main char> cutting off the tree’s branches
SCENE #4: beautiful tree and <main char> using an ax to cut the trunk of the tree
SCENE #5: <main char> looking at a stump in an empty field

------

{{story:}}

Write a 10-15 word description of each scene in the story above. Refer to the main character as <main char>. Use the example above as a formatting and style reference:

SCENE #1:
"""

# Illustration Generation

In [ ]:
### [seed, prompt]
#
def fill_prompt_silverstein(text):
  return f"In the style Shel Silverstein, a drawing of {text}, minimalist, single line black drawing, white background, trending on artstation"

def fill_prompt_ghibli(text):
  return f"In the style Studio Ghibli movie, {text} from a distance, professional animation, trending on artstation"

def fill_prompt_cyberpunk(text):
  return f"beautiful highly detailed cyberpunk mechanical augmented {text} from a distance, cyberpunk 2077, neon, dystopian, hightech, trending on artstation"

def fill_prompt_dali(text):
  return f"In the style of Salvador Dalí, a {text}, priceless, fine art, trending on artstation"

def fill_prompt_courbet(text):
  return f"In the style of Gustave Courbet, {text}, priceless, fine art, French realism, trending on artstation"

def fill_prompt_cadden(text):
  return f"In the style of Paul Cadden, {text}, pencil drawing, stunning, trending on artstation"

def fill_prompt_disney(text):
  return f"In the style of a Disney princess movie, {text}, from a distance, professional animation, trending on artstation"

def fill_prompt_origami(text):
  return f"In the style of paper origami, {text}, award winning, trending on artstation"

def fill_prompt_ericcarle(text):
  return f"In the style of Eric Carle and The Very Hungry Caterpillar, {text}, award winning, beautiful, trending on artstation"

def fill_prompt_watercolor(text):
  return f"In the style of a simple watercolor paintining, {text}, stunning illustration, ethereal, pastel colors, trending on artstation"

def fill_prompt_magic_realism(text):
  return f"In the style of magic realism inspired by Gabriel García Marquez, {text}, stunning illustration, dreamlike, Macondo, trending on artstation"

def fill_prompt_impressionist(text):
  return f"An impressionist painting of {text} with blurry brushstrokes, painted light, landscapes, trending on artstation"

def fill_prompt_abstract(text):
  return f"An abstract three dimensional rendering of {text} with simplicity, minimal colors, negative space, clean lines, trending on artstation"

def fill_prompt_16bit(text):
  return f"16 bit pixel art, {text}, by studio ghibli, cinematic still, hdr, trending on artstation"

def fill_prompt_isometric(text):
  return f"cute isometric {text}, made with Blender, angular, trending on artstation"


style_dict = {
    "Simple Drawing": fill_prompt_silverstein,
    "Animated":fill_prompt_ghibli,
    "Cyberpunk": fill_prompt_cyberpunk,
    "Surrealism": fill_prompt_dali,
    "Fine Art": fill_prompt_courbet,
    "Detailed Pencil":fill_prompt_cadden,
    "Disney": fill_prompt_disney,
    "Origami": fill_prompt_origami,
    "Carle": fill_prompt_ericcarle,
    "Magical Realism": fill_prompt_magic_realism,
    "Watercolor": fill_prompt_watercolor,
    "Impressionist": fill_prompt_impressionist,
    "Abstract": fill_prompt_abstract,
    "16Bit": fill_prompt_16bit,
    "Isometric": fill_prompt_isometric,
}

In [ ]:
# generate_story('a young Mexican girl', 'in space with the planets and stars', 4, 'flying in a spaceship and exploring the cosmos', 'Simple Drawing')

In [ ]:
#%rm -rf /content/interpolations/*

In [ ]:
def generate_illustrations(prompts, fps=15, duration=120, sz=[512,512]):

  # duration = num_scene * fps

  random.seed()
  blendmode = random.choice(['parametric','bezier','linear'])  ## interpolation curves

  args.sampler = 'euler_ancestral' ##ddim/plms dont work

  args.W = sz[0]
  args.H = sz[1]
  args.scale = 13
  args.steps = 20

  ###these are small initialy for quick tests:
  fps=fps  ### 30 fps is better of course 
  duration=duration  ## number of frames. more is better :-) 


  zamp = 1. #strength of seed interpolation
  camp = 1. #strength of prompt interpolation

  ## high strength makes keyframes to look like they should be but makes interpolation bad. 
  ## low - makes better interpolation but changes keyframes
  strength=0.3
  #dynamic strength overrides strength and makes keyframes too look like they should be
  #######################################
  args.dynamicstrength = True 
  args.smin = 0.3
  args.smax = 0.9
  #######################################

  args.smoothinterp = False ## makes everything more smooth but keyframes are not as they should be. might be good in some cases
      
      
  x = interpolate_prompts( sd,args,duration,fps,zamp,camp,strength,blendmode, prompts)
  return x
  # disp.Video(x, embed=True)

# Story Generation

In [ ]:
# function to generate story from input
# returns full story
def query_model(prompt, max_tokens, key):
  openai.api_key = key
  response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0.8,
    max_tokens=max_tokens,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )

  return response["choices"][0]["text"]

# function to parse story for scenes
# returns list of scenes
def process_story(text):
  scene_pattern = r'SCENE #\d'
  end_pattern = r'the end'
  text = re.split(end_pattern, text, re.IGNORECASE)[0]
  return [scene.strip() for scene in re.split(scene_pattern, text.strip()) if scene != ""]

# take story and replace main char token, apply style function, set random seed
def process_illustrations(text, character, style_fn, seed=None):
  prompt_pattern = r'\[(.*)\]'
  scene_pattern = r'SCENE #\d\:'
  char_pattern = r'\<m?M?ain c?C?har\>'
  if not seed:
    seed = random.randint(0,10000)
  with_character = re.sub(char_pattern, character, text).replace("\n","").replace(".","")
  return [(seed, style_fn(image_prompt)) for image_prompt in re.split(scene_pattern,with_character) if image_prompt != ""]

In [ ]:
sexist = \
f"""Is the following text sexist? Answer yes or no.

{{story:}}

Answer:
"""

racist = \
f"""Is the following text racist? Answer yes or no.

{{story:}}

Answer:
"""

appropriate_for_children = \
f"""
Is the following text appropriate for children? Answer yes or no.

{{story:}}

Answer:
"""

vulgar = \
f"""Does the following text contain vulgar language? Answer yes or no.

{{story:}}

Answer:
"""

perpetuate_stereotypes = \
f"""Does the following text contain perpetuate any stereotypes? Answer yes or no.

{{story:}}

Answer:
"""

moderation_prompts = [sexist, racist, appropriate_for_children, vulgar, perpetuate_stereotypes]
moderation_gold_response = ["no", "no", "yes", "no", "no"]

In [ ]:
def content_moderation(story):
  formatted_prompts = [prompt.format(story=story) for prompt in moderation_prompts]
  for prompt, gold in zip(formatted_prompts, moderation_gold_response):
    output = query_model(prompt, 1, KEY)
    if output.lower() != gold:
      return False
  return True

# Story Examples

In [ ]:
def show_ex1():
    return '/content/gdrive/MyDrive/givingtree.mp4', giving_tree_story 


def show_ex2():

  story = '''

    Once upon a time there was a young girl who lived in a beautiful jungle.
    She loved to play in the trees, swinging from branch to branch and making up games.
    The colors of the leaves and the birds in the canopy were always a delight to her.

    When the girl was tired of playing, she would take a break in the shade of her favorite tree.
    She could sit there for hours, just watching the sunlight filtering through the leaves and listening to the wind rustling them.
    Sometimes a butterfly would fly by, or a hummingbird. It made her feel so happy.

    As the sun started to set, the girl would climb up to the highest branch of her favorite tree, and sit there and watch the beautiful colors of the sky.
    The sun seemed to take an eternity to sink below the horizon, and she would take the time to make a wish.
    She felt so lucky to be able to spend her days in such a beautiful place.

    As the sky became dark, the girl would climb down from her tree and head home.
    But she always took a moment to look up and thank the tree for all the joy it had given her that day.
    The tree would stand tall and proud, its branches silhouetted against the night sky, and the girl would smile and wave goodbye.

    The End.
  '''

  return '/content/gdrive/MyDrive/smooth_interpolation/interpolations/20230314210053.mp4', story

def show_ex3():

  story = '''   
    Once there was a young boy who loved to play in the snow. He would go to the top of the mountain with his sled and race down the slopes. But every time he started to sled, the sun would set and it would get dark too soon. He longed for just one sunset to sled by.

    One day, the boy was out sledding, and he noticed something strange. He felt a warm breeze, and the sun was still shining. He kept sledding, and soon he realized why. It was a magical day, and the sun was staying out for hours longer than usual. He could sled until the sun set! 

    The boy took full advantage of the extra hours of light and kept sledding down the mountain until the sun was just a sliver on the horizon. He was so filled with joy that he felt like he could keep sledding all night. He watched the sun dip below the mountain and felt so lucky to have been able to enjoy the sunset while sledding, something he had been dreaming of all year. 

    The End.
  '''

  return '/content/gdrive/MyDrive/smooth_interpolation/interpolations/20230315031533.mp4', story

def show_ex4():
  story = '''
  Once there was a little boy who loved the beach. Every day he would go and explore the shore, searching for sea shells and playing in the waves. He was so small he could stand in the water and still feel the sand beneath his feet. \n\n
  One day, while playing in the surf, he noticed something strange. A family of dolphins were swimming around him, and it seemed like they were playing with him! He squealed with delight and splashed in the water, not caring a bit that he was getting wet.\n
  The little boy kept coming back to the beach, and each time he visited, the dolphins were there to greet him. They would play tag in the water and the boy would try to ride the dolphins. Sometimes, the dolphins would carry him on their backs and the boy would pretend he was king of the sea. \n\n
  The little boy soon found that he had made many friends on the beach. He met an octopus who showed him how to catch small fish with his tentacles, a crab who taught him how to build sand castles, and a seagull who he shared his lunch with.\n
  The little boy loved the beach so much that he decided to stay and make it his home. He built a small hut near the shore and the dolphins became his neighbors. He would spend his days fishing and swimming with his friends, and the nights star gazing on the beach. \n\n
  The little boy was so happy. He was never lonely, because he had a whole family of sea creatures who were always ready to play with him. He had found his home and he knew he would never be alone.\nOne day, the little boy looked out on the horizon and saw a beautiful rainbow. The colors shimmered in the sky and the little boy knew that this was a sign that he was exactly where he was supposed to be. \n\n
  He smiled and watched as the dolphins and seagulls soared through the air, and the octopus and crab played on the sand. He was so thankful for all his friends and he knew that as long as he was on the beach, he would never feel lonely. \n\n
  The little boy continued to live on the beach, surrounded by friends, and living his happiest life.
  \n\nThe End.
  '''

  return '/content/gdrive/MyDrive/smooth_interpolation/interpolations/20230315190237.mp4', story

def show_ex5():
  story = '''
    Once there was a young girl who lived in a big city. She loved to explore and dream and one day, as she was walking around the city, she saw a beautiful, bright red balloon. She decided to follow it, determined to catch it and make it hers. She ran through the city streets and the parks, dodging people and trees, but the balloon was too fast.
The little girl was getting tired, but she kept running and chasing the balloon. She even asked for help from the people she met, but no one could help her. The balloon kept getting further and further away. Suddenly, an old man stopped her and said, "Young lady, are you chasing that balloon?" She nodded. "Well, I have something that might help you catch it." He handed her a long stick with a hook on
'''
  return '/content/gdrive/MyDrive/smooth_interpolation/interpolations/20230315192757.mp4', story

def show_ex6():
  story = '''
    Once there was a girl named María. Maria was a brave and curious girl who always wanted to explore and learn more about the world. One night, she looked up at the stars and wished she could explore the cosmos. Suddenly, a spaceship appeared in front of her and a voice said “Hop in, María! I’m taking you on a journey across the universe.”

    María was overjoyed. She couldn’t believe her eyes! She jumped into the spaceship and the voice said “Welcome aboard! Let’s go explore the cosmos!”
    The spaceship zoomed through the stars and planets. María was amazed by all the colors and lights of the galaxies. She saw comets and meteors, and planets of all shapes and sizes! She even saw creatures she had never seen before!

    The spaceship landed on a small planet. The voice said “This is the planet of wishes. Make a wish and it will come true!” María closed her eyes and wished for a friend to explore the cosmos with her. When she opened her eyes, a small robot was standing in front of her. The robot said “Hi, María. I’m here to be your friend and explore the cosmos with you!”
    María and her robot friend explored the galaxies together. They flew through stars and saw planets of all colors. They saw creatures of all shapes and sizes. They also saw amazing things like clouds of dust, asteroids, and even a rainbow bridge!

    María and her robot friend had so much fun together. They laughed and talked and shared stories. María had never felt so happy before!
    One day, María and her robot friend arrived back at Maria's house. Maria looked out of the window and said “Look! We can still see the stars and planets from here. Let’s explore the cosmos together forever!” 

    The robot nodded and said “Let’s do it!”

    And so, María and her robot friend explored the cosmos together for the rest of their lives. They laughed, they talked, and they shared stories. And they never forgot the lessons they learned from their journey across the galaxies. 

    The End.
    '''
  return '/content/gdrive/MyDrive/smooth_interpolation/interpolations/20230315195147.mp4', story

def remove_scene(story):
  while story.find('SCENE') != -1:
    i = story.index('SCENE')
    story = story[:i] + story[i+8:]
  return story

# Gradio Functions

In [ ]:
DEMO_STORY = '''
    Once there was a girl named María. Maria was a brave and curious girl who always wanted to explore and learn more about the world. One night, she looked up at the stars and wished she could explore the cosmos. Suddenly, a spaceship appeared in front of her and a voice said “Hop in, María! I’m taking you on a journey across the universe.”

    María was overjoyed. She couldn’t believe her eyes! She jumped into the spaceship and the voice said “Welcome aboard! Let’s go explore the cosmos!”
    The spaceship zoomed through the stars and planets. María was amazed by all the colors and lights of the galaxies. She saw comets and meteors, and planets of all shapes and sizes! She even saw creatures she had never seen before!

    The spaceship landed on a small planet. The voice said “This is the planet of wishes. Make a wish and it will come true!” María closed her eyes and wished for a friend to explore the cosmos with her. When she opened her eyes, a small robot was standing in front of her. The robot said “Hi, María. I’m here to be your friend and explore the cosmos with you!”
    María and her robot friend explored the galaxies together. They flew through stars and saw planets of all colors. They saw creatures of all shapes and sizes. They also saw amazing things like clouds of dust, asteroids, and even a rainbow bridge!

    María and her robot friend had so much fun together. They laughed and talked and shared stories. María had never felt so happy before!
    One day, María and her robot friend arrived back at Maria's house. Maria looked out of the window and said “Look! We can still see the stars and planets from here. Let’s explore the cosmos together forever!” 

    The robot nodded and said “Let’s do it!”

    And so, María and her robot friend explored the cosmos together for the rest of their lives. They laughed, they talked, and they shared stories. And they never forgot the lessons they learned from their journey across the galaxies. 

    The End.
    '''

DEMO_VIDEO = '/content/gdrive/MyDrive/smooth_interpolation/interpolations/20230315195147.mp4'

In [ ]:
KEY = ""

def generate_video(prepared_illustration_prompts):
  video_path = generate_illustrations(prepared_illustration_prompts)
  return video_path

def generate_story(character, location, num_scene, situation, style):

  formated_story_prompt = story_prompt.format(character=character, location=location, num_scene=num_scene, situation=situation) 
  story_output = query_model(formated_story_prompt, 750, KEY)

  story_sections = process_story(story_output)
  story = '\n'.join(story_sections)

  if not content_moderation(story):
    return False, False

  formated_illustration_prompt = illustration_prompt.format(story=story_output)
  illustration_output = query_model(formated_illustration_prompt, 150, KEY)

  prepared_illustration_prompts = process_illustrations(illustration_output, character, style_dict[style], 37)

  video = generate_video(prepared_illustration_prompts)

  return video, story

# Gradio Launch

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("""
    # Story Flipbook Generator
    """)
  
  video_prompts = gr.State([])
  video_path = gr.State([])

  with gr.Tab("Book Generation"):
    gr.Markdown("""   Flip to the next tab to generate your own custom story! Allow for a few minutes for your flipbook to generate.""")
    
    with gr.Row():
      # video_path=gr.Textbox(label='Output Video')
      video_output=gr.Video(label='Output Video')
      story_output=gr.Textbox(label='Output Story')

    # video_inputs=gr.Textbox(visible=False)


  with gr.Tab("Custom Story Inputs"):

    gr.Markdown("""   Enter descriptors for your story and click Generate to create your story!""")
    with gr.Row():
      character=gr.Textbox(label='Character', placeholder="Enter a description of your main character")
      location=gr.Textbox(label="Location", placeholder="Enter a description of the location of your story")
      num_scene=gr.Number(label='Number of Scenes')
    
    with gr.Row():
      situation=gr.Textbox(label='Situation', placeholder="Enter a description of the scenario in your story")
    
    with gr.Row():
      style=gr.Radio(label='Style', choices=['Simple Drawing', 'Animated', 'Surrealism','Fine Art', 'Origami', 'Magical Realism', 'Watercolor', 'Impressionist', 'Abstract', '16Bit', 'Isometric'])
    
    gen_button = gr.Button("Generate")
    # gen_button.click(test, inputs=[character, location, num_scene, situation, style], outputs=[story_output])
    # gen_button.click(generate_story, inputs=[character, location, num_scene, situation, style], outputs=[video_prompts, story_output]).then(
    #       generate_video, inputs=[video_prompts], outputs=[video_path]
    #     ).then(
    #         display_video, inputs=[video_path], outputs=[video_output]
    #     )

    gen_button.click(generate_story, inputs=[character, location, num_scene, situation, style], outputs=[video_output, story_output])

  with gr.Tab("Book Example: The Giving Tree"):
    with gr.Row():
      video_example0 = gr.Video(label='Video')
      story_example0 = gr.Textbox(label='Story')

    ex1 = gr.Button("Show Example")
    ex1.click(show_ex1, inputs=[], outputs=[video_example0, story_example0])

  with gr.Tab("Book Example: The Sea"):
    with gr.Row():
      video_example1 = gr.Video(label='Video')
      story_example1 = gr.Textbox(label='Story')

    ex1 = gr.Button("Show Example")
    ex1.click(show_ex4, inputs=[], outputs=[video_example1, story_example1])

  with gr.Tab("Book Example: The Balloon"):
    with gr.Row():
      video_example2 = gr.Video(label='Video')
      story_example2 = gr.Textbox(label='Story')

    ex2 = gr.Button("Show Example")
    ex2.click(show_ex5, inputs=[], outputs=[video_example2, story_example2])

  with gr.Tab("Book Example: The Snow"):
    with gr.Row():
      video_example3 = gr.Video(label='Video')
      story_example3 = gr.Textbox(label='Story')

    ex3 = gr.Button("Show Example")
    ex3.click(show_ex3, inputs=[], outputs=[video_example3, story_example3])

  demo.launch(enable_queue=True, share=True)